In [2]:
import torch

print(torch.cuda.device_count())  # 使用可能なGPUの数を表示
print(torch.cuda.get_device_name(0))  # 最初のGPUの名前を表示
print(torch.cuda.get_device_name(1))  # 二番目のGPUの名前を表示

2
NVIDIA RTX A6000
NVIDIA RTX A6000


In [3]:
import torch
import logging
device = torch.device("cuda")
logger = logging.getLogger(__name__)

### デフォルト自作関数

In [8]:
# 保存用
import pickle
def read_bin(filename):
    with open(filename,'rb') as bf:
        bin_data = pickle.load(bf)
    return bin_data

def save_bin(filename,data):
    with open(filename,'wb') as bf:
        pickle.dump(data,bf)

In [9]:
def print_memory_usage():
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i)
        reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} - Allocated memory: {allocated / 1024 ** 2:.2f} MB")
        print(f"GPU {i} - Reserved memory: {reserved / 1024 ** 2:.2f} MB")

## GPT models

### GPT-2 

In [7]:
# 役立たず
# from transformers import GPT2Tokenizer, GPT2LMHeadModel

# model_name = "gpt2"
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# model = GPT2LMHeadModel.from_pretrained(model_name)

### GPT-NeoX

In [8]:
# !pip install transformers

In [32]:
# from transformers import AutoTokenizer, GPTNeoXForCausalLM

# model_name = "EleutherAI/gpt-neox-20b"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = GPTNeoXForCausalLM.from_pretrained(model_name)

In [4]:
# saved_model_name = "saved_model/gpt-neox"
# model.save_pretrained(saved_model_name)
# tokenizer.save_pretrained(saved_model_name)

In [7]:
from transformers import AutoTokenizer, GPTNeoXForCausalLM

saved_model_name = "saved_model/gpt-neox"
model=GPTNeoXForCausalLM.from_pretrained(saved_model_name)
print("Model loaded successfully")

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

Model loaded successfully


In [12]:
model.half()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 6144)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-43): 44 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=6144, out_features=18432, bias=True)
          (dense): Linear(in_features=6144, out_features=6144, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=6144, out_features=24576, bias=True)
          (dense_4h_to_h): Linear(in_features=24576, out_features=6144, bias=T

In [13]:
import torch
import logging
device = torch.device("cuda")
logger = logging.getLogger(__name__)

model = torch.nn.DataParallel(model,device_ids=[0, 1])  # モデルをデータ並列に設定
torch.cuda.empty_cache()
model.to(device)

DataParallel(
  (module): GPTNeoXForCausalLM(
    (gpt_neox): GPTNeoXModel(
      (embed_in): Embedding(50432, 6144)
      (emb_dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-43): 44 x GPTNeoXLayer(
          (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
          (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
          (post_attention_dropout): Dropout(p=0.0, inplace=False)
          (post_mlp_dropout): Dropout(p=0.0, inplace=False)
          (attention): GPTNeoXSdpaAttention(
            (rotary_emb): GPTNeoXRotaryEmbedding()
            (query_key_value): Linear(in_features=6144, out_features=18432, bias=True)
            (dense): Linear(in_features=6144, out_features=6144, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (mlp): GPTNeoXMLP(
            (dense_h_to_4h): Linear(in_features=6144, out_features=24576, bias=True)
            (dense

In [14]:
print_memory_usage()

GPU 0 - Allocated memory: 39391.02 MB
GPU 0 - Reserved memory: 39396.00 MB
GPU 1 - Allocated memory: 0.00 MB
GPU 1 - Reserved memory: 0.00 MB


In [16]:
tokenizer=AutoTokenizer.from_pretrained(saved_model_name)
print("tokenizer loaded successfully")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer loaded successfully


In [18]:
print_memory_usage()
torch.cuda.empty_cache()

GPU 0 - Allocated memory: 39391.02 MB
GPU 0 - Reserved memory: 39396.00 MB
GPU 1 - Allocated memory: 0.00 MB
GPU 1 - Reserved memory: 0.00 MB


In [15]:
torch.cuda.empty_cache()

### llama1

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "baffo32/decapoda-research-llama-7B-hf"  # 初期のLLaMAモデルの名前に置き換えてください
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

In [43]:
def neox_ans(tweet,question):
    prompt = "Read the following tweet and evaluate the question provided. Return the answer as Yes, No, or Unknown. Tweet:"+tweet+" Question:"+ question+"."
#     print(prompt)
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    prompt_length = input_ids.shape[1]
    # print(prompt_length)
    torch.cuda.empty_cache()
    output = model.module.generate(
        input_ids,
        pad_token_id=tokenizer.eos_token_id,
        max_length=prompt_length+50
    )
    ans = tokenizer.decode(output[0], skip_special_tokens=True)
    return ans

### 24/7/8

## データ収集

In [19]:
nlp4if_b21 = read_bin("new_data/nlp4if_b21.bin")
# CT22_b21 = read_bin("new_data/CT22_b21.bin")
FABLE = read_bin("new_data/FABLE.bin")
# Important_Question_flag = read_bin("new_data/Important_Question_flag.bin")

In [20]:
nlp4if_b21[0]

{'tweet_no': 505,
 'text': "India's gift of 100,000 COVID-19 vaccines arrived Barbados earlier today. This was a very special moment for all Barbadians and I want to thank Prime Minister Modi for his quick, decisive, and magnanimous action in allowing us to be the beneficiary of these vaccines. https://t.co/cSCb40c2mt",
 'q1_label': 'yes',
 'q2_label': 'no',
 'q3_label': 'yes',
 'q4_label': 'no',
 'q5_label': 'no',
 'q6_label': 'no',
 'q7_label': 'no',
 'tweet_id': 1359350943356174337}

In [21]:
print_memory_usage()
torch.cuda.empty_cache()

GPU 0 - Allocated memory: 39391.02 MB
GPU 0 - Reserved memory: 39396.00 MB
GPU 1 - Allocated memory: 0.00 MB
GPU 1 - Reserved memory: 0.00 MB


In [ ]:
# tweet_no_list = []

In [ ]:
i = 3
for d in nlp4if_b21[i:100]:
    tweet = d["text"]
    ans_list=[]
    ans_list.append(d["tweet_no"])

    j = 1
    for question in FABLE:
        print('\r%d / %d  | %d / %d' %(j,len(FABLE),i,len(nlp4if_b21)), end='')
        ans = neox_ans(tweet,question)
        ans_list.append(ans)
        j += 1
    save_bin("new_data/nlp4if_b21_ans/"+str(d["tweet_no"])+".bin",ans_list)
    tweet_no_list.append(d["tweet_no"])
    
    i += 1

33 / 52  | 11 / 418

In [32]:
tweet_no_list

[505, 506, 507]

In [33]:
data_505 = read_bin("new_data/nlp4if_b21_ans/"+str(tweet_no_list[0])+".bin")

In [34]:
data_505

[505,
 "Read the following tweet and evaluate the question provided. Return the answer as Yes, No, or Unknown. Tweet:India's gift of 100,000 COVID-19 vaccines arrived Barbados earlier today. This was a very special moment for all Barbadians and I want to thank Prime Minister Modi for his quick, decisive, and magnanimous action in allowing us to be the beneficiary of these vaccines. https://t.co/cSCb40c2mt Question:Does the message fit into a larger story or argument, for example about how the world works or how people think?.\n\nThe answer is Yes. The message fits into a larger story or argument about how the world works or how people think. The message is about how the world works. The message is about how people think. The message is about how the world",
 "Read the following tweet and evaluate the question provided. Return the answer as Yes, No, or Unknown. Tweet:India's gift of 100,000 COVID-19 vaccines arrived Barbados earlier today. This was a very special moment for all Barbadia

In [ ]:
test = 

In [ ]:
CT22_b21_ans = []
for d in CT22:
    tweet = d["tweet_text"]
    ans_list.append(d["tweet_id"])
    for question in FABLE:
        ans = neox_ans(tweet,question)
        ans_list.append(ans)

In [ ]:
save_bin("new_data/FABLE.bin",FABLE)

In [17]:
tweet = CT22[7]["tweet_text"]
question = FABLE[10]

In [19]:
ans = neox_ans(tweet,question)
ans

Read the following tweet and evaluate the question provided. Return the answer as Yes, No, or Unknown. Tweet:Those who falsely claim Vaccines are 100% safe and who laugh at anti-vaxxers will be dependent on those who refuse to tell the official lies to say that wide-spread vaccination *is* critical. 23 die in Norway after receiving COVID-19 vaccine: https://t.co/eN10HwZvqW via @nypost Question:Does the message content include any threats of violence?.


'Read the following tweet and evaluate the question provided. Return the answer as Yes, No, or Unknown. Tweet:Those who falsely claim Vaccines are 100% safe and who laugh at anti-vaxxers will be dependent on those who refuse to tell the official lies to say that wide-spread vaccination *is* critical. 23 die in Norway after receiving COVID-19 vaccine: https://t.co/eN10HwZvqW via @nypost Question:Does the message content include any threats of violence?.\n\nThe correct answer is Yes.\n\nThe tweet contains a threat of violence.\n\nThe tweet is a threat of violence.\n\nThe tweet is not a threat of violence.\n\nThe tweet is not a threat of violence.\n\nThe tweet is not a threat of'

In [17]:
n = 'Read the following tweet and evaluate the question provided. Return the answer as Yes, No, or Unknown. Tweet:WSJ: All three of Russia\'s main intelligence services (SVR, FSB, and GRU) ran fake news sites and phony social media accounts to spread misinformation about Pfeizer and Moderna vaccines in an attempt to undermine confidence in them. https://t.co/t3kmCYvBFs via @WSJ Question:Does the message question the trustworthiness of other people in general within a community or society?\n\nThe answer is Yes.\n\nThe message is a form of social proof. It is a way of saying, "Look at all these people who agree with me'

In [18]:
n

'Read the following tweet and evaluate the question provided. Return the answer as Yes, No, or Unknown. Tweet:WSJ: All three of Russia\'s main intelligence services (SVR, FSB, and GRU) ran fake news sites and phony social media accounts to spread misinformation about Pfeizer and Moderna vaccines in an attempt to undermine confidence in them. https://t.co/t3kmCYvBFs via @WSJ Question:Does the message question the trustworthiness of other people in general within a community or society?\n\nThe answer is Yes.\n\nThe message is a form of social proof. It is a way of saying, "Look at all these people who agree with me'

### 日付選別